## Segmenting and Clustering Neighborhoods in Toronto
### Before we get the data and start exploring it, let's download all the dependencies that we will need.

### ***Please Note that all questions of the data captsone is answered in this notebook***

In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#!pip install -U scikit-learn scipy matplotlib
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

## Question 1
### The following codes scrape the url for the toronto data , then Data Wrangling is carried out

In [2]:
import requests
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
#!pip install Beautifulsoup
from bs4 import BeautifulSoup
torontolink=BeautifulSoup(link,'html.parser')
print(torontolink.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrRJvgpAEKcAAI@-EI0AAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":955414546,"wgRevisionId":955414546,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [4]:
My_table = torontolink.find('table',{'class':'wikitable sortable'})

In [5]:
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East Y

In [6]:
print(My_table.tr.text)


Postal Code

Borough

Neighborhood



In [7]:
headers="Postcode,Borough,Neighbourhood"

table=[]
for tr in My_table.find_all('tr'):
    row1=[]
    for tds in tr.find_all('td'):
       # row1=row1+","+tds.text
        row1.append(tds.text)
    table.append(row1)
    #table=table+row1[1:]
print(table)

[[], ['M1A\n', 'Not assigned\n', '\n'], ['M2A\n', 'Not assigned\n', '\n'], ['M3A\n', 'North York\n', 'Parkwoods\n'], ['M4A\n', 'North York\n', 'Victoria Village\n'], ['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n'], ['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n'], ['M7A\n', 'Downtown Toronto\n', "Queen's Park, Ontario Provincial Government\n"], ['M8A\n', 'Not assigned\n', '\n'], ['M9A\n', 'Etobicoke\n', 'Islington Avenue\n'], ['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n'], ['M2B\n', 'Not assigned\n', '\n'], ['M3B\n', 'North York\n', 'Don Mills\n'], ['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n'], ['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n'], ['M6B\n', 'North York\n', 'Glencairn\n'], ['M7B\n', 'Not assigned\n', '\n'], ['M8B\n', 'Not assigned\n', '\n'], ['M9B\n', 'Etobicoke\n', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n'], ['M1C\n', 'Scarborough\n', 'Rouge Hill, Port Union, Highland Creek\n'], ['M

In [93]:
#file1= open("toronto.csv","wb")
#file.write(bytes(headers,encoding="ascii",errors="ignore"))
#file1.write(bytes(table,encoding="ascii",errors="ignore"))
import pandas as pd
df=pd.DataFrame(table)


In [94]:
df.head()

,0,1,2
0,None,None,None
1,M1A\n,Not assigned\n,\n
2,M2A\n,Not assigned\n,\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


In [95]:

#df = pd.read_csv('toronto.csv',header=None,error_bad_lines=False)
df.columns=["Postalcode","Borough","Neighbourhood"]
df.head()

,Postalcode,Borough,Neighbourhood
0,None,None,None
1,M1A\n,Not assigned\n,\n
2,M2A\n,Not assigned\n,\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


In [96]:
#df.dropna(axis=0)
notindex= df[df['Borough']== 'Not assigned\n'].index
df.drop(notindex,inplace=True)

In [97]:
df['Borough']=df['Borough'].astype(str)
df['Postalcode']=df['Postalcode'].astype(str)
df['Neighbourhood']=df['Neighbourhood'].astype(str)


In [98]:
df['Borough']=df['Borough'].apply(lambda row: row.replace(' ', '')[:-1])
df['Postalcode']=df['Postalcode'].apply(lambda row: row.replace(' ', '')[:-1])
df['Neighbourhood']=df['Neighbourhood'].apply(lambda row: row.replace(' ', '')[:-1])
df.head()


,Postalcode,Borough,Neighbourhood
0,Non,Non,Non
3,M3A,NorthYork,Parkwoods
4,M4A,NorthYork,VictoriaVillage
5,M5A,DowntownToronto,"RegentPark,Harbourfront"
6,M6A,NorthYork,"LawrenceManor,LawrenceHeights"


In [99]:
df.reset_index(drop=True, inplace=True)
df=df.drop([0])

In [101]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']

In [102]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

In [106]:
dff=result.reset_index()
dff.head(10)

,Postalcode,Borough,Neighbourhood
0,M3A,NorthYork,Parkwoods
1,M4A,NorthYork,VictoriaVillage
2,M5A,DowntownToronto,"RegentPark,Harbourfront"
3,M6A,NorthYork,"LawrenceManor,LawrenceHeights"
4,M7A,DowntownToronto,"Queen'sPark,OntarioProvincialGovernment"
5,M9A,Etobicoke,IslingtonAvenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,NorthYork,DonMills
8,M4B,EastYork,"ParkviewHill,WoodbineGardens"
9,M5B,DowntownToronto,"GardenDistrict,Ryerson"


In [108]:
dff.shape

(103, 3)

## Question 2
### Read in Geospatial csv and merge with toronto neighborhood csv and Display

In [109]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0ec375c6e9f24767a5a83b89b0d652f0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='MTJelLZWyg2juS97tidoNkN82jOGH0RCOiEixSf52yrh',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_0ec375c6e9f24767a5a83b89b0d652f0.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-tc3zwde4iyik7m',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [110]:
df_data_0.columns=['Postalcode','Latitude','Longitude']

In [111]:
Toronto_df = pd.merge(dff,
                 df_data_0[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
Toronto_df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,VictoriaVillage,43.725882,-79.315572
2,M5A,DowntownToronto,"RegentPark,Harbourfront",43.654260,-79.360636
3,M6A,NorthYork,"LawrenceManor,LawrenceHeights",43.718518,-79.464763
4,M7A,DowntownToronto,"Queen'sPark,OntarioProvincialGovernment",43.662301,-79.389494
5,M9A,Etobicoke,IslingtonAvenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
7,M3B,NorthYork,DonMills,43.745906,-79.352188
8,M4B,EastYork,"ParkviewHill,WoodbineGardens",43.706397,-79.309937
9,M5B,DowntownToronto,"GardenDistrict,Ryerson",43.657162,-79.378937


## Question 3 Toronto Neighborhood Clustering

In [112]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [113]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [114]:
CLIENT_ID = 'UW0OQLUDAFYXL4JW5OYZW2VVWD4MWRQXC1SXZ4FW44IMGJZE' # your Foursquare ID
CLIENT_SECRET = 'QDWVW13UVJRIJNM23G4Y4OEU3ZKOT05MPJ0KCB5YVQIY44ZU' # your Foursquare Secret
VERSION = '20191225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UW0OQLUDAFYXL4JW5OYZW2VVWD4MWRQXC1SXZ4FW44IMGJZE
CLIENT_SECRET:QDWVW13UVJRIJNM23G4Y4OEU3ZKOT05MPJ0KCB5YVQIY44ZU


In [115]:
radius=500
LIMIT=100

In [116]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [117]:

toronto_venues = getNearbyVenues(names=Toronto_df['Neighbourhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Parkwoods
VictoriaVillage
RegentPark,Harbourfront
LawrenceManor,LawrenceHeights
Queen'sPark,OntarioProvincialGovernment
IslingtonAvenue
Malvern,Rouge
DonMills
ParkviewHill,WoodbineGardens
GardenDistrict,Ryerson
Glencairn
WestDeanePark,PrincessGardens,MartinGrove,Islington,Cloverdale
RougeHill,PortUnion,HighlandCreek
DonMills
WoodbineHeights
St.JamesTown
Humewood-Cedarvale
Eringate,BloordaleGardens,OldBurnhamthorpe,MarklandWood
Guildwood,Morningside,WestHill
TheBeaches
BerczyPark
Caledonia-Fairbanks
Woburn
Leaside
CentralBayStreet
Christie
Cedarbrae
HillcrestVillage
BathurstManor,WilsonHeights,DownsviewNorth
ThorncliffePark
Richmond,Adelaide,King
Dufferin,DovercourtVillage
ScarboroughVillage
Fairview,HenryFarm,Oriole
NorthwoodPark,YorkUniversity
EastToronto
HarbourfrontEast,UnionStation,TorontoIslands
LittlePortugal,Trinity
KennedyPark,Ionview,EastBirchmountPark
BayviewVillage
Downsview
TheDanforthWest,Riverdale
TorontoDominionCentre,DesignExchange
Brockton,ParkdaleVillage,ExhibitionPla

In [118]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,VictoriaVillage,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,VictoriaVillage,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [119]:
toronto_venues.shape

(2148, 7)

In [121]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood,LongBranch",10,10,10,10,10,10
"BathurstManor,WilsonHeights,DownsviewNorth",18,18,18,18,18,18
BayviewVillage,4,4,4,4,4,4
"BedfordPark,LawrenceManorEast",24,24,24,24,24,24
BerczyPark,57,57,57,57,57,57
"BirchCliff,CliffsideWest",4,4,4,4,4,4
"Brockton,ParkdaleVillage,ExhibitionPlace",23,23,23,23,23,23
BusinessreplymailProcessingCentre,16,16,16,16,16,16


### Analysing each of the Toronto Neighborhoods

In [122]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [123]:
toronto_onehot.shape

(2148, 267)

In [124]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000

### Printing each neighborhood along with the top 5 most common venues

In [ ]:

num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Alderwood,LongBranch----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2             Gym   0.1
3    Skating Rink   0.1
4  Sandwich Place   0.1


----BathurstManor,WilsonHeights,DownsviewNorth----
            venue  freq
0            Bank  0.11
1     Coffee Shop  0.11
2        Pharmacy  0.06
3  Ice Cream Shop  0.06
4   Shopping Mall  0.06


----BayviewVillage----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----BedfordPark,LawrenceManorEast----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Place  0.08
4                 Pub

               venue  freq
0              Field  0.25
1       Hockey Arena  0.25
2              Trail  0.25
3               Park  0.25
4  Health Food Store  0.00


----IndiaBazaar,TheBeachesWest----
                  venue  freq
0                  Park  0.15
1  Fast Food Restaurant  0.10
2    Italian Restaurant  0.05
3        Ice Cream Shop  0.05
4         Burrito Place  0.05


----KennedyPark,Ionview,EastBirchmountPark----
              venue  freq
0    Discount Store  0.29
1  Department Store  0.14
2        Hobby Shop  0.14
3       Bus Station  0.14
4       Coffee Shop  0.14


----KensingtonMarket,Chinatown,GrangePark----
                   venue  freq
0                   Café  0.09
1            Coffee Shop  0.07
2  Vietnamese Restaurant  0.05
3     Mexican Restaurant  0.05
4                 Bakery  0.05


----KingsviewVillage,St.Phillips,MartinGroveGardens,RichviewGardens----
               venue  freq
0        Pizza Place  0.25
1     Sandwich Place  0.25
2           Bus Line  0.25
